# 1. Set Environments

In [1]:
import wandb
import os
os.environ["WANDB_PROJECT"]="Machin Translator_01"

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aeolian83. Use `wandb login --relogin` to force relogin


True

In [2]:
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()


login(token= os.environ["HF_TOKEN"])

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/aeolian83/.cache/huggingface/token
Login successful


# 2. Set Datasets

In [3]:
from datasets import Dataset, DatasetDict
import pickle
import os
import time


with open('./data/train_data_300.pkl', 'rb') as file:
    train_data = pickle.load(file)
len(train_data)

with open('./data/validation_data_28.pkl', 'rb') as file:
    test_data = pickle.load(file)
len(test_data)

train_dataset = Dataset.from_list(train_data)
test_dataset = Dataset.from_list(test_data)

# DatasetDict로 "train"과 "test" 데이터셋 묶기
dataset_dict = DatasetDict({
        'train': train_dataset,
        'test': test_dataset
    })

# 3. Checkpoints Listing

In [4]:
checkpoint_dir = "./checkpoint/translate_machine_llama3ko_intsuct_origindata300_02"

# 디렉터리 이름에서 숫자를 추출하는 함수
def extract_number(directory_name):
    return int(directory_name.split('-')[-1])


# 디렉터리 항목을 숫자 순서대로 정렬
checkpoints = []
with os.scandir(checkpoint_dir) as entries:
    for entry in entries:
        if entry.is_dir():
            checkpoints.append(entry.name)

# 숫자 순서대로 정렬
checkpoints.sort(key=extract_number)

# 4. Evaluate

### (1) Function

In [5]:
import evaluate
import re
import pandas as pd
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch

metric = evaluate.load("sacrebleu")

model_id = "beomi/Llama-3-KoEn-8B-Instruct-preview"
device_map = {"": 0}
cache_model_dir="/mnt/t7/.cache/huggingface/models"

def make_prompt(text):
    return f'''Translate input sentence to Korean
### Input: {text}
### Translated:'''


def clean_text(text):
    text = re.sub(r'\(.*?\)', '', text)
    return text.strip()

def count_mask(batchs):
    mask = []
    for batch in batchs["attention_mask"]:
        count = torch.sum(batch == 0).item()
        mask.append(count)

    return mask

def counter_terms(terms, text):
    if not isinstance(terms, list):
        terms = terms.split(", ")

    return sum(text.lower().count(term.lower()) for term in terms)

def generate(inputs, tokenizer, model):
    examples = []

    for input in inputs:
        prompt = make_prompt(input)
        examples.append(prompt)

    example_batch = tokenizer(examples, return_tensors="pt", padding=True).to(model.device)

    mask = count_mask(example_batch)

    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**example_batch, max_new_tokens = 512, pad_token_id=tokenizer.pad_token_id)

    outputs = [tokenizer.decode(t[len(tokenizer.encode(examples[i])) + mask[i]:], skip_special_tokens=True) for i, t in enumerate(output_tokens)]

    return outputs


def evaluate_score(datasets, outputs):
    if not len(datasets["korean"]) == len(outputs):
        pass

    inputs = datasets["english"]
    labels = datasets["korean"]
    terms = datasets["terms"]

    results = []

    for input, output, label, term in zip(inputs, outputs, labels, terms):
        
        label_clean = clean_text(label)
        prediction_clean = clean_text(output)
        result = metric.compute(predictions=[prediction_clean], references=[label_clean])
        print(f"Label: {label_clean}")
        print(f"Output: {prediction_clean}")
        print("*" * 100)

        input_count = counter_terms(term, input)
        predic_count = counter_terms(term, output)

        weight = 1.0 if predic_count > input_count else predic_count / input_count

        results.append((result['score'], weight, weight * result['score']))
    
    return results


model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=BitsAndBytesConfig(load_in_8bit=True), device_map='auto', cache_dir=cache_model_dir)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

### (2) Evaluate Model

In [6]:
# 시작 시간 기록
start_time = time.time()

bleu_scores = {}

for checkpoint in checkpoints[10:]:
    print('#' * 50)
    print(checkpoint)
    print('#' * 50)

    peft_model_id = os.path.join(checkpoint_dir, checkpoint)
    tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
    tokenizer.padding_side = "left"
    model.resize_token_embeddings(len(tokenizer))

    loaded_model = PeftModel.from_pretrained(model=model, model_id=peft_model_id)

    results = []
    for i in range(0, len(dataset_dict["test"]), 6):
        dataset = dataset_dict["test"][i:i+6]

        outputs = generate(dataset["english"], tokenizer, loaded_model)

        results += evaluate_score(dataset, outputs)

    bleus, weights, weighted_bleus = zip(*results)
    score = {
        "bleu": np.mean(bleus),
        "weight": np.mean(weights),
        "weighted_bleus": np.mean(weighted_bleus)
        }
    bleu_scores[checkpoint] = score

    del peft_model_id, tokenizer, loaded_model
    torch.cuda.empty_cache()

df = pd.DataFrame.from_dict(bleu_scores, orient='index')

# 종료 시간 기록
end_time = time.time()

print(f"Execution Time: {end_time - start_time} seconds")
print(df)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


##################################################
checkpoint-110
##################################################
Label: 그룹 희소성은 다중선형 대수에서 변수 그룹 내의 희소성 패턴을 촉진하는 개념입니다. 이 기술은 특히 고차원 데이터와 관련된 응용 분야에서 유용하며, 관련된 특징 그룹을 식별하는 데 도움을 줍니다. 팩터 그래프에서 그룹 희소성은 그래프 구조의 복잡성을 줄여 추론 알고리즘의 효율성을 높일 수 있습니다. 다중선형 대수는 이러한 그룹 간의 상호작용을 이해하고 조작하는 수학적 기초를 제공합니다. 그룹 희소성과 다중선형 대수를 활용함으로써 팩터 그래프는 대규모 문제를 더 효과적으로 처리할 수 있습니다.
Output: 그룹 희소성은 변수 집합 내에서 희소 패턴을 촉진하는 다변수 대수의 개념입니다. 이 기술은 고차원 데이터를 다루는 응용 프로그램에서 특히 유용하며, 관련된 특징 집합을 식별하는 데 도움을 줍니다. 인자 그래프에서 그룹 희소성은 그래프 구조의 복잡성을 줄임으로써 추론 알고리즘의 효율성을 높일 수 있습니다. 다변수 대수는 이러한 그룹 간의 상호작용을 이해하고 조작하는 수학적 기반을 제공합니다. 그룹 희소성과 다변수 대수를 활용하여 인자 그래프는 대규모 문제를 더 효과적으로 처리할 수 있습니다.
****************************************************************************************************
Label: 잠재 공간 모델은 데이터를 저차원 공간에서 표현하는 강력한 도구입니다. 이러한 모델은 데이터 내의 숨겨진 구조를 발견하기 위해 잠재 변수 추론에 의존합니다. 그러나 정확한 잠재 변수 추론은 계산 비용이 많이 들 수 있기 때문에, 근사 추론 방법이 자주 사용됩니다. 근사 추론 기법은 철저한 계산 없이 잠재 변수를 추정하는 실용적인 방법을 제공합니다. 근사 추론을 활용

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Label: 그룹 희소성은 다중선형 대수에서 변수 그룹 내의 희소성 패턴을 촉진하는 개념입니다. 이 기술은 특히 고차원 데이터와 관련된 응용 분야에서 유용하며, 관련된 특징 그룹을 식별하는 데 도움을 줍니다. 팩터 그래프에서 그룹 희소성은 그래프 구조의 복잡성을 줄여 추론 알고리즘의 효율성을 높일 수 있습니다. 다중선형 대수는 이러한 그룹 간의 상호작용을 이해하고 조작하는 수학적 기초를 제공합니다. 그룹 희소성과 다중선형 대수를 활용함으로써 팩터 그래프는 대규모 문제를 더 효과적으로 처리할 수 있습니다.
Output: 그룹 희소성은 변수 집합 내에서 희소성 패턴을 촉진하는 다변수 대수의 개념입니다. 이 기술은 고차원 데이터를 다루는 응용 프로그램에서 특히 유용하며, 관련된 특징 집합을 식별하는 데 도움을 줍니다. 인자 그래프 맥락에서 그룹 희소성은 그래프 구조의 복잡성을 줄임으로써 추론 알고리즘의 효율성을 향상시킬 수 있습니다. 다변수 대수는 이러한 그룹 간의 상호작용을 이해하고 조작하는 수학적 기반을 제공합니다. 그룹 희소성과 다변수 대수를 활용함으로써 인자 그래프는 대규모 문제를 더 효과적으로 처리할 수 있도록 최적화될 수 있습니다.
****************************************************************************************************
Label: 잠재 공간 모델은 데이터를 저차원 공간에서 표현하는 강력한 도구입니다. 이러한 모델은 데이터 내의 숨겨진 구조를 발견하기 위해 잠재 변수 추론에 의존합니다. 그러나 정확한 잠재 변수 추론은 계산 비용이 많이 들 수 있기 때문에, 근사 추론 방법이 자주 사용됩니다. 근사 추론 기법은 철저한 계산 없이 잠재 변수를 추정하는 실용적인 방법을 제공합니다. 근사 추론을 활용함으로써, 잠재 공간 모델은 대규모 데이터셋과 복잡한 데이터 구조를 효율적으로 처리할 수 있습니다.
Output: 잠재 공간 모델은 데이터를 저차원 공간에 표현하는 강력한 도구

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

Label: 그룹 희소성은 다중선형 대수에서 변수 그룹 내의 희소성 패턴을 촉진하는 개념입니다. 이 기술은 특히 고차원 데이터와 관련된 응용 분야에서 유용하며, 관련된 특징 그룹을 식별하는 데 도움을 줍니다. 팩터 그래프에서 그룹 희소성은 그래프 구조의 복잡성을 줄여 추론 알고리즘의 효율성을 높일 수 있습니다. 다중선형 대수는 이러한 그룹 간의 상호작용을 이해하고 조작하는 수학적 기초를 제공합니다. 그룹 희소성과 다중선형 대수를 활용함으로써 팩터 그래프는 대규모 문제를 더 효과적으로 처리할 수 있습니다.
Output: 그룹 희소성은 변수 집합 내에서 희소성 패턴을 촉진하는 다변수 대수의 개념입니다. 이 기법은 고차원 데이터를 포함하는 응용 프로그램에서 특히 유용하며, 관련된 특징 집합을 식별하는 데 도움을 줍니다. 인자 그래프에서 그룹 희소성은 그래프 구조의 복잡성을 줄여 추론 알고리즘의 효율성을 향상시킬 수 있습니다. 다변수 대수는 이러한 그룹 간의 상호작용을 이해하고 조작하는 수학적 기초를 제공합니다. 그룹 희소성과 다변수 대수를 활용함으로써 인자 그래프는 대규모 문제를 더 효과적으로 처리할 수 있도록 최적화될 수 있습니다.
****************************************************************************************************
Label: 잠재 공간 모델은 데이터를 저차원 공간에서 표현하는 강력한 도구입니다. 이러한 모델은 데이터 내의 숨겨진 구조를 발견하기 위해 잠재 변수 추론에 의존합니다. 그러나 정확한 잠재 변수 추론은 계산 비용이 많이 들 수 있기 때문에, 근사 추론 방법이 자주 사용됩니다. 근사 추론 기법은 철저한 계산 없이 잠재 변수를 추정하는 실용적인 방법을 제공합니다. 근사 추론을 활용함으로써, 잠재 공간 모델은 대규모 데이터셋과 복잡한 데이터 구조를 효율적으로 처리할 수 있습니다.
Output: 잠재 공간 모델은 데이터를 저차원 공간에 표현하는 강력한 기계 학습 도

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Label: 그룹 희소성은 다중선형 대수에서 변수 그룹 내의 희소성 패턴을 촉진하는 개념입니다. 이 기술은 특히 고차원 데이터와 관련된 응용 분야에서 유용하며, 관련된 특징 그룹을 식별하는 데 도움을 줍니다. 팩터 그래프에서 그룹 희소성은 그래프 구조의 복잡성을 줄여 추론 알고리즘의 효율성을 높일 수 있습니다. 다중선형 대수는 이러한 그룹 간의 상호작용을 이해하고 조작하는 수학적 기초를 제공합니다. 그룹 희소성과 다중선형 대수를 활용함으로써 팩터 그래프는 대규모 문제를 더 효과적으로 처리할 수 있습니다.
Output: 그룹 희소성은 변수 집합 내에서 희소성 패턴을 촉진하는 다변수 대수의 개념입니다. 이 기술은 고차원 데이터를 처리하는 응용 프로그램에서 특히 유용하며, 관련된 특징 집합을 식별하는 데 도움을 줍니다. 인자 그래프에서는 그룹 희소성이 그래프 구조의 복잡성을 줄여 추론 알고리즘의 효율성을 향상시킬 수 있습니다. 다변수 대수는 이러한 그룹 간의 상호작용을 이해하고 조작하는 수학적 기초를 제공합니다. 그룹 희소성과 다변수 대수를 활용함으로써 인자 그래프는 대규모 문제를 더 효과적으로 처리할 수 있습니다.
****************************************************************************************************
Label: 잠재 공간 모델은 데이터를 저차원 공간에서 표현하는 강력한 도구입니다. 이러한 모델은 데이터 내의 숨겨진 구조를 발견하기 위해 잠재 변수 추론에 의존합니다. 그러나 정확한 잠재 변수 추론은 계산 비용이 많이 들 수 있기 때문에, 근사 추론 방법이 자주 사용됩니다. 근사 추론 기법은 철저한 계산 없이 잠재 변수를 추정하는 실용적인 방법을 제공합니다. 근사 추론을 활용함으로써, 잠재 공간 모델은 대규모 데이터셋과 복잡한 데이터 구조를 효율적으로 처리할 수 있습니다.
Output: 잠재 공간 모델은 데이터를 저차원 공간에 표현하는 강력한 도구입니다. 이러한 모델은 잠재

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Label: 그룹 희소성은 다중선형 대수에서 변수 그룹 내의 희소성 패턴을 촉진하는 개념입니다. 이 기술은 특히 고차원 데이터와 관련된 응용 분야에서 유용하며, 관련된 특징 그룹을 식별하는 데 도움을 줍니다. 팩터 그래프에서 그룹 희소성은 그래프 구조의 복잡성을 줄여 추론 알고리즘의 효율성을 높일 수 있습니다. 다중선형 대수는 이러한 그룹 간의 상호작용을 이해하고 조작하는 수학적 기초를 제공합니다. 그룹 희소성과 다중선형 대수를 활용함으로써 팩터 그래프는 대규모 문제를 더 효과적으로 처리할 수 있습니다.
Output: 그룹 희소성은 변수 집합 내에서 희소성 패턴을 촉진하는 다변수 대수의 개념입니다. 이 기술은 고차원 데이터를 처리하는 응용 프로그램에서 특히 유용하며, 관련된 특징 집합을 식별하는 데 도움을 줍니다. 인자 그래프에서는 그룹 희소성이 그래프 구조의 복잡성을 줄여 추론 알고리즘의 효율성을 향상시킬 수 있습니다. 다변수 대수는 이러한 그룹 간의 상호작용을 이해하고 조작하는 수학적 기초를 제공합니다. 그룹 희소성과 다변수 대수를 활용하여 인자 그래프는 대규모 문제를 더 효과적으로 처리할 수 있습니다.
****************************************************************************************************
Label: 잠재 공간 모델은 데이터를 저차원 공간에서 표현하는 강력한 도구입니다. 이러한 모델은 데이터 내의 숨겨진 구조를 발견하기 위해 잠재 변수 추론에 의존합니다. 그러나 정확한 잠재 변수 추론은 계산 비용이 많이 들 수 있기 때문에, 근사 추론 방법이 자주 사용됩니다. 근사 추론 기법은 철저한 계산 없이 잠재 변수를 추정하는 실용적인 방법을 제공합니다. 근사 추론을 활용함으로써, 잠재 공간 모델은 대규모 데이터셋과 복잡한 데이터 구조를 효율적으로 처리할 수 있습니다.
Output: 잠재 공간 모델은 데이터를 저차원 공간에 표현하는 강력한 도구입니다. 이러한 모델은 데이터 